In [1]:
import nltk 
import string 
import re 
import pandas as pd
import boto3
import pickle
from sklearn.neighbors import KDTree
import io
import numpy as np

import sagemaker
from sagemaker import get_execution_role
import json

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = 'inroads-blazingtext' # Replace with your own bucket name if needed
print(bucket)
prefix = 'blazingtext/supervised'

arn:aws:iam::068949824886:role/service-role/AmazonSageMaker-ExecutionRole-20200617T154104
inroads-blazingtext


In [2]:
#download labeled data

In [66]:
s3 = boto3.client('s3')

object = s3.get_object(Bucket='inroads-test-bucket1',Key='data/labeled_data.csv')
labeled_data = pd.read_csv(io.BytesIO(object['Body'].read()), encoding='utf8', index_col=0)

labeled_data.shape

(10146, 2)

In [67]:
#format for blazing text

In [33]:
train_op_data=[]
for i in range(0,len(labeled_data)):
    label = "_label_" + str(labeled_data['cluster'][i])
    train_op_data.append(label)
len(train_op_data)
train_op_data

['_label_2',
 '_label_2',
 '_label_3',
 '_label_1',
 '_label_4',
 '_label_3',
 '_label_3',
 '_label_2',
 '_label_1',
 '_label_4',
 '_label_1',
 '_label_3',
 '_label_2',
 '_label_5',
 '_label_2',
 '_label_3',
 '_label_2',
 '_label_3',
 '_label_1',
 '_label_5',
 '_label_8',
 '_label_1',
 '_label_5',
 '_label_2',
 '_label_8',
 '_label_4',
 '_label_4',
 '_label_4',
 '_label_4',
 '_label_3',
 '_label_4',
 '_label_3',
 '_label_4',
 '_label_4',
 '_label_4',
 '_label_4',
 '_label_3',
 '_label_4',
 '_label_4',
 '_label_3',
 '_label_4',
 '_label_4',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_4',
 '_label_4',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_8',
 '_label_0',
 '_label_0',
 '_label_8',
 '_label_3',
 '_label_3',
 '_label_3',
 '_label_1',
 '_label_4',
 '_label_4',
 '_label_4',
 '_label_1',
 '_label_3',
 '_label_1',
 '_label_2',
 '_label_5',
 '_label_2',
 '_label_1',
 '_label_8',
 '_label_3',
 '_label_2',
 '_label_1',

In [51]:
labeled_data['op_data']=train_op_data
txt=labeled_data['op_data']+" "+labeled_data['token']
txt.head()
#txt.to_csv('train.csv',index=False)

0     _label_2 abstract
1      _label_2 absence
2    _label_3 abundance
3      _label_1 reality
4    _label_4 beautiful
dtype: object

In [68]:
# create training and testing vars
txt_copy = txt.copy()
train_set = txt_copy.sample(frac=0.75, random_state=0)
test_set = txt_copy.drop(train_set.index)

train_set.to_csv('train.csv', index=False)
test_set.to_csv('validation.csv', index=False)

In [7]:
## format 

In [69]:
region_name = boto3.Session().region_name

container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Using SageMaker BlazingText container: 811284229777.dkr.ecr.us-east-1.amazonaws.com/blazingtext:latest (us-east-1)


In [70]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

In [71]:
%%time

train_channel = prefix + '/train'
validation_channel = prefix + '/validation'


sess.upload_data(path='train.csv',bucket=bucket, key_prefix=train_channel)
sess.upload_data(path='validation.csv', bucket=bucket, key_prefix=validation_channel)

s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)

CPU times: user 45.9 ms, sys: 0 ns, total: 45.9 ms
Wall time: 337 ms


In [72]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

In [73]:
bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.c4.4xlarge',
                                         train_volume_size = 30,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [74]:
bt_model.set_hyperparameters(mode="supervised",
                            epochs=10,
                            min_count=2,
                            learning_rate=0.05,
                            vector_dim=10,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=1)

In [77]:
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/plain', s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [78]:
bt_model.fit(inputs=data_channels, logs=False)


2020-08-25 19:15:03 Starting - Starting the training job
2020-08-25 19:15:06 Starting - Launching requested ML instances....................
2020-08-25 19:16:49 Starting - Preparing the instances for training........
2020-08-25 19:17:37 Downloading - Downloading input data....
2020-08-25 19:17:59 Training - Downloading the training image..
2020-08-25 19:18:15 Training - Training image download completed. Training in progress..
2020-08-25 19:18:28 Uploading - Uploading generated training model
2020-08-25 19:18:28 Failed - Training job failed


UnexpectedStatusException: Error for Training job blazingtext-2020-08-25-19-15-03-439: Failed. Reason: ClientError: Training did not complete successfully! Please check the logs for errors.

In [15]:
text_classifier = bt_model.deploy(initial_instance_count = 1,instance_type = 'ml.t2.medium')
Use JSON format for inference

0     _label_2 abstract
1      _label_2 absence
2    _label_3 abundance
3      _label_1 reality
4    _label_4 beautiful
dtype: object

In [ ]:
## for lambda

In [ ]:
import boto3
runtime = boto3.Session().client('sagemaker-runtime')
print(runtime)

In [ ]:
print(text_classifier.endpoint)